## Example to load the network, sample 4 levels at random from the latent space and then plot them using matplotlib.


In [54]:
# from google.colab import drive
# drive.mount('/content/drive')

In [55]:
#!cp 'path/to/file' 'path/to/drive'

In [56]:
# import os
# os.chdir("lab6/exercise_DL_pcg")

In [57]:
!dir

 Il volume nell'unit� D � Windows10_2
 Numero di serie del volume: 36D1-2B32

 Directory di d:\ITU\DeepLearningLabs\deep-learning-for-games-labs\lab6\exercise_DL_pcg

19/09/2022  10:46    <DIR>          .
19/09/2022  10:46    <DIR>          ..
15/09/2021  20:28        46.794.090 all_levels_onehot.npz
15/09/2021  21:16               821 example_sampling_random_levels.py
19/09/2022  10:27    <DIR>          models
15/09/2021  21:11             2.243 plotting_utilities.py
15/09/2021  21:18             1.003 readme.md
15/09/2021  21:17               255 requirements.txt
19/09/2022  10:27    <DIR>          sprites
15/09/2021  20:47             4.526 train_vae.py
15/09/2021  21:09             1.966 vae_mario.py
19/09/2022  10:46    <DIR>          __pycache__
               7 File     46.804.904 byte
               5 Directory  225.643.634.688 byte disponibili


In [58]:
import os
import torch
import matplotlib.pyplot as plt

from vae_mario import VAEMario
from plotting_utilities import plot_decoded_level

In [59]:
# Loading the model
model_name = "mario_vae_zdim_2_overfitted"
z_dim = 2
vae = VAEMario(z_dim=z_dim)
vae.load_state_dict(torch.load(f"./models/{model_name}.pt"))


VAEMario(
  (encoder): Sequential(
    (0): Linear(in_features=2156, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
  )
  (fc_mu): Sequential(
    (0): Linear(in_features=256, out_features=2, bias=True)
  )
  (fc_var): Sequential(
    (0): Linear(in_features=256, out_features=2, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=2, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=2156, bias=True)
  )
)


<All keys matched successfully>

In [68]:
# Sampling random zs
zs = 2.5 * torch.randn((4, z_dim))
levels = vae.decode(zs)
level_1 = levels[0]
target = torch.argmax(level_1, dim=0)
target

tensor([[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [2, 2, 2, 2, 1, 2, 2, 2, 2, 3, 2, 1, 1, 1],
        [2, 2, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1],
        [2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1],
        [2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [84]:
# z=torch.randn(z_dim)
# output = torch.argmax(vae.decode(z)[0], dim=0)
# Create a matrix of 0 and 1 to show which parts are different
# compare_tensor = (output == target).int()
# _,wrong_tiles = compare_tensor.unique(return_counts=True)
# print(wrong_tiles[0])
# Create some z in order to find the best fit for the previous target


zs = [z + 0.1 * torch.randn(z_dim) for z in range(10)]
zs = torch.stack(zs)

def find_best_z(target, zs):
    levels = vae.decode(zs)
    best_match = 10000
    z = 0
    for idx, level in enumerate(levels):
        differences = 0
        output = torch.argmax(level, dim=0)
        compare_tensor = (output == target).int()
        _,count = compare_tensor.unique(return_counts=True)
        wrong_tiles = count[0]
        if wrong_tiles < best_match:
            z = zs[idx]
            best_match = wrong_tiles
            print("Found better fitting, new wrong tiles: ", best_match)
    return z, best_match

def improve_z(target, z):
    zs = [z + 0.15 * torch.randn(z_dim) for _ in range(100000)]
    zs = torch.stack(zs)
    return find_best_z(target, zs)

z, best = find_best_z(target, zs)
print(best)
new_z, new_best = improve_z(target, z)
print(new_best)


Found better fitting, new wrong tiles:  tensor(79)
tensor(79)
Found better fitting, new wrong tiles:  tensor(79)
Found better fitting, new wrong tiles:  tensor(77)
Found better fitting, new wrong tiles:  tensor(74)
Found better fitting, new wrong tiles:  tensor(72)
Found better fitting, new wrong tiles:  tensor(71)
tensor(71)


In [61]:
# Getting levels from them using the decoder
levels = vae.decode(zs)
# print(levels.shape)
level_imgs = [plot_decoded_level(level) for level in levels]


In [62]:
# Plotting
_, axes = plt.subplots(1, 4, figsize=(7 * 4, 7))
for level_img, ax in zip(level_imgs, axes):
    ax.imshow(level_img)
    ax.axis("off")

plt.tight_layout()
plt.show()


# Latent Variable Evolution Lab


## Sample from the VAE and then implement some search algorithm of choise to search the latent space for a particular level (e.g. one with many ground tiles)